In [7]:
import xarray as xr
import dask

**Make sure to first download the SeasFire dataset following the details in the readme**

In [2]:
# Configuration

# path to original dataset
ds_path = # Add path to original dataset (E.g. ./seasfire_0.25deg.zarr)

# path to output dataset (coarsened)
ds_coarse_path = # Add path to coarsened dataset (E.g. ./seasfire_1deg.zarr)

# coarsening factor (4 -> from 0.25 degree to 1 degree)
coarse_factor = 4 

In [3]:
ds = xr.open_zarr(ds_path)

In [9]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_coarse = ds.coarsen(latitude=4, longitude=4).mean()

    sum_vars = ['area',
    'cams_co2fire',
    'cams_frpfire',
    'fcci_ba',
    'fcci_fraction_of_burnable_area',
    'fcci_fraction_of_observed_area',
    'fcci_number_of_patches',
    'gfed_ba',
    'gwis_ba']
    for var in sum_vars:
        ds_coarse[var] = ds[var].coarsen(latitude=4, longitude=4).sum()

    mode_vars = ['gfed_region']
    for var in mode_vars:
        ds_coarse[var] = ds[var].coarsen(latitude=4, longitude=4).median()

In [13]:
chunk_dict = {'time': -1, 'latitude': -1, 'longitude': -1}

for var in ds_coarse:
    if ds_coarse[var].encoding.get('chunks'):
        del ds_coarse[var].encoding['chunks']
    if ds_coarse[var].encoding.get('preferred_chunks'):
        del ds_coarse[var].encoding['preferred_chunks']
for var in ds_coarse.coords:
    if ds_coarse[var].encoding.get('chunks'):
        del ds_coarse[var].encoding['chunks']
    if ds_coarse[var].encoding.get('preferred_chunks'):
        del ds_coarse[var].encoding['preferred_chunks']
ds_coarse.chunk(chunk_dict).to_zarr(output_path, consolidated=True)